# SIP、多媒体、IP电话网络流量分析

包括IP电话、视频会议、视频监控以及在线教育在内的多类多媒体应用是现代化网络通信的重要组成部分。

这些多媒体应用对网络的要求与前面介绍的众多企业网应用有所不同。

像HTTP、EMAIL、FTP等企业应用需要较高的带宽，而IP电话等多媒体应用需要的带宽并不高（只需要100kbit/s，高清视频的会高些），但对网络延迟及抖动（延迟的差异）却很敏感。

对于那些拥有远程分支机构的企业，在各分支结构内，绝大多数企业网应用都会引发价高的下载流量，而诸如视频监控系统之类的多媒体应用却会触发较高的上传流量。

本章会关注包括语音、视频在内的多媒体应用，以及它们在网络中的运作方式，同时还会分析一些故障。



## wireshark内置IP电话和多媒体流量分析工具

### 监听位置

监听IP电话、视频电话会议或其他多媒体会话应用时，要在交换机上开启镜像，监听流量。

若要监听整个IP电话网络故障（IP PBX故障），需要对连接IP PBX的端口设置镜像。PBX(Private Branch eXchange,专用交换机）简而言之就是集团电话，将电话网与计算机网统一成一个整体，它被广泛地运用在企业办公机构中，极大地提高了企业的办事效率。

### wireshark telephony工具

![wireshark电话菜单](images/11/wireshark电话菜单.png)

## SIP协议分析

SIP（Session Initiation Protocol，会话初始协议）是一个基于文本的应用层控制协议，用于创建、修改和释放一个或多个参与者的会话。可以支持并应用于语音、视频、数据等多媒体业务，同时也可以应用于Presence（呈现）、Instant Message（即时消息）等特色业务。可以说，有IP网络的地方就有SIP协议的存在。

SIP由IETF制定的多媒体通信协议,是主要的视频会议标准之一（另一个是H.323).

目前，大规模商用VoIP网络和会议系统都采用H.323标准，但随着VoIP技术的进一步发展和视频会议系统应用的逐渐普及，SIP协议正日益受到业界的重视，基于SIP的视频会议系统获得越来越广泛的应用。可以说，有IP网络的地方就有SIP协议的存在。

> H.323和SIP分别是通信领域与因特网两大阵营推出的协议。H.323企图把IP电话当作是众所周知的传统电话，只是传输方式发生了改变，由电路交换变成了分组交换。而SIP协议侧重于将IP电话作为因特网上的一个应用，较其它应用（如FTP，E-mail等）增加了信令和QoS的要求，它们支持的 业务基本相同，也都利用RTP作为媒体传输的协议。但H.323是一个相对复杂的协议。

> H.323采用基于ASN.1和压缩编码规则的二进制方法表示其消息。ASN.1通常需要特殊的代码生成器来进行词法和语法分析。而SIP的基于文本的协议，类似于HTTP。基于文本的编码意味着头域的含义是一目了然的，如From、To、Subject等域名。这种分布式、几乎不需要复杂的文档说明的标准规范风格，其优越性已在过去的实践中得到了充分的证明（如今广为流行的邮件协议SMTP就是 这样的一个例子）。SIP的消息体部份采用SDP进行描述，SDP中的每一项格式为=，也比较简单。

> 在支持会议电话方面，H.323由于由多点控制单元（MCU）集中执行会议控制功能，所有参加会议终端都向MCU发送控制消息，MCU可能会成为瓶颈，特别是对于具有附加特性的大型会议；并且H.323不支持信令的组播功能，其单功能限制了可扩展性，降低了可靠性。而SIP设计上就为分布式的呼叫模型，具有分布式的组播功能，其组播功能不仅便于会议控制，而且简化了用户定位、群组邀请等，并且能节约带宽。但是H.323的集中控制便于计费，对带宽的管理也比较简单、有效。

**案例一：ZOOM会议系统同时采用了H.323和SIP：**

![zoom会议图](images/11/zoom会议图.png)

**案例二：京东与Genesys携手打造的高可靠、易扩展、更开放的新一代客服中心：**

解决的问题：
- 稳定、容错、灾备方案放在第一位，性能和时效第二位；
- 建设高可靠性，扩展性，开放性的新一代客服中心；
- 需要支撑5000席位以上，日呼叫量50万-80万的呼叫中心平台，支持至少2000席的远程座席，用作开放平台的商户座席；
- 融合语音、视频、数据业务，引入适应京东未来业务发展的新媒体应用。

![jd-sip呼叫中心项目系统拓扑1](images/11/jd-sip呼叫中心项目系统拓扑1.jpg)

![jd-sip呼叫中心项目系统拓扑2](images/11/jd-sip呼叫中心项目系统拓扑2.jpg)

### 相关术语

#### 用户代理（User Agent）

SIP的会话端点被称为用户代理。

创建SIP会话的过程，实际上就是UA之间进行协商，对会话参数达成一致意见的过程。

#### 服务器

提供定位会话参与者、注册、呼叫转发、其它服务的网络主机，叫服务器。

UA向服务器发送注册请求消息、会话邀请请求、以及其他请求。

服务器则分为以下类型：

- SIP代理服务器（Proxy Server）：作用是先接收来自UA或其他代理服务器的SIP控制请求消息，再转发或代为转发至其它位置。

- SIP重定向服务器（Redirect Server）：作用是接收来自UA或其它代理服务器的SIP控制请求消息，向他们回复包含状态码3xx（重定向）的SIP响应消息，指明SIP控制请求消息实际的归宿。

- SIP注册服务器（Register）：作用是接受来自用户的SIP注册请求，将用户的位置信息登记进名为“位置服务”（location service）的数据库（或其他数据库）。

#### 呼叫

一个呼叫是由一个公共源端所邀请的在一个会议中的所有参加者组成，有一个全球唯一的Call-ID进行标识。

例如：由同一个源邀请的一个会议的所有参加者，构成一个呼叫；点到点的IP电话会话是最简单的会话，它映射为单一的SIP呼叫。

通常情况下，呼叫由主叫方创建，也可由并不参与媒体通信的第三方创建（会话的主教方和邀请方并不相同）。

对于多播会议，一个用户可由不同的人邀请参加同一个会议，则每一个邀请视为不同的呼叫。

#### 事务

SIP是一个C/S服务协议。客户和服务器之间的操作从第1个请求，至最终响应为止的所有消息构成一个SIP事务。

一个正常的呼叫，一般包含三个事务：

- 呼叫启动的邀请（invite）；

- 呼叫启动的证实（ACK），证实已收到最终响应；

- 呼叫终结，包含一个操作：再见（Bye）

#### SIP URL

为了能正确传送协议消息，SIP还需要解决两个重要的问题：

- 寻址

  + 采用SIP URL，用户名字段可以是电话号码，以支持IP电话网关寻址，实现IP与PSTN的互通。 
  + 通常由SIP:开头

- 用户定位

  + 基于登记实现。SIP用户终端上电后，即向服务器登记（register）。

某种程度上，SIP与HTTP类似。

#### 在同一域中建立SIP会话的过程

1. 用户 A 发起此呼叫，告诉 SIP 代理服务器要联系用户 B；

2. SIP 代理服务器向 SIP 注册服务器发出请求，要求提供用户 B 的 IP 地址；

3. 收到 B 的 SIP 地址, SIP收到用户 B 的 IP 地址

4. SIP 代理服务器转发用户 A 与用户 B 进行通信的邀请信息（使用 SDP），包括用户 A 要使用的媒体（线路资源）；

5. 用户 B 通知 SIP 代理服务器可以接受用户 A 的邀请，且已做好接收消息的准备；

6. SIP 代理服务器将此消息传达给用户 A，从而建立 SIP 会话；

7. 用户创建一个点到点 RTP 连接，实现用户间的交互通信。

下面是一个更为详细的通信过程流程图：

![SIP通信过程](images/11/SIP通信过程.jpg)

### SIP 协议格式

SIP协议的亮点却不在于它的强大，而是在于：简单。

SIP协议是一个Client/Sever协议，因此SIP消息分两种：

- 请求消息，是SIP客户端为了激活特定操作而发给服务器端的消息；
- 响应消息，SIP协议中的响应消息用于对请求消息进行响应，指示呼叫的成功或失败状态。

#### 常用的SIP请求消息

- INVITE：表示主叫用户发起会话请求，邀请其他用户加入一个会话。也可以用在呼叫建立后用于更新会话（此时该INVITE又称为Re-invite）。
- ACK：客户端向服务器端证实它已经收到了对INVITE请求的最终响应。
- PRACK：表示对1xx响应消息的确认请求消息。
- BYE：表示终止一个已经建立的呼叫。
- CANCEL：表示在收到对请求的最终响应之前取消该请求，对于已完成的请求则无影响。
- REGISTER：表示客户端向SIP服务器端注册列在To字段中的地址信息。
- OPTIONS：表示查询被叫的相关信息和功能。

#### 常用的一些响应消息：

- 100试呼叫（Trying）
- 180振铃（Ringing）
- 181呼叫正在前转（Call is Being Forwarded）
- 200成功响应（OK）
- 302临时迁移（Moved Temporarily）
- 400错误请求（Bad Request）
- 401未授权（Unauthorized）
- 403禁止（Forbidden）
- 404用户不存在（Not Found）
- 408请求超时（Request Timeout）
- 480暂时无人接听（Temporarily Unavailable）
- 486线路忙（Busy Here）
- 504服务器超时（Server Time-out）
- 600全忙（Busy Everywhere）

### 本节实验操作

- 打开样本sample_sip_rtp.pcap文件
- 打开菜单“电话”-“SIP 统计（SIP statistcs）”，回答以下问题：
- 该样本的SIP请求有哪些类型？分别有多少个？
- 该样本的响应消息有哪些类型？分别有多少个？

- 打开菜单“电话”-“SIP 流”；
- 样本中有几个会话？
- 对于第26分组和第18分组，填写以下表格：

|分组号|开始时间（精确到小数后2位）|结束时间（精确到小数后2位）|源SIP URL|目的SIP URL|持续时间|状态|注释|
|-|-|-|-|-|-|-|-|
|26|
|18|

- SIP协议基于哪种传输层协议工作？

- 打开样本文件MagicJack_short_call.pcap 文件
- 打开菜单“电话”-“SIP 流”；
- 打开“流序列”，观察并总结通信会话过程。  
  + 例如：第一步， 192.168.0.10 邀请216.234.64.8，invite sdp；
  + 第二步，216.234.64.8应答192.168.0.10，响应码为100，意思是试呼叫；
  + ...
  
- 使用“播放流”功能，监听其中的通信内容，将监听到的内容写在实验报告里。


## RTP 协议分析

RTP协议用于在互联网上传递音频和视频的标准数据包。

RTP协议常用于流媒体系统（配合RTSP协议），视频会议和一键通（Push to Talk）系统（配合H.323或SIP），使它成为IP电话产业的技术基础。

RTP协议和RTP控制协议RTCP一起使用，而且它是创建在UDP协议上的。它一开始被设计为一个多播协议，但后来被用在很多单播应用中。

实时传输协议（RTP）为数据提供了具有实时特征的端对端传送服务，如在组播或单播网络服务下的交互式视频音频或模拟数据。应用程序通常在 UDP 上运行 RTP 以便使用其多路结点和校验服务；这两种协议都提供了传输层协议的功能。但是 RTP 可以与其它适合的底层网络或传输协议一起使用。如果底层网络提供组播方式，那么 RTP 可以使用该组播表传输数据到多个目的地。

RTP 本身并没有提供按时发送机制或其它服务质量（QoS）保证，它依赖于底层服务去实现这一过程。 RTP 并不保证传送或防止无序传送，也不确定底层网络的可靠性。 RTP 实行有序传送， RTP 中的序列号允许接收方重组发送方的包序列，同时序列号也能用于决定适当的包位置，例如：在视频解码中，就不需要顺序解码。

### RTP典型的应用场合

#### 简单的多播音频会议

语音通信通过一个多播地址和一对端口来实现。一个用于音频数据（RTP），另一个用于控制包（RTCP）。

#### 音频和视频会议

如果在一次会议中同时使用了音频和视频会议，这两种媒体将分别在不同的RTP会话中传送，每一个会话使用不同的传输地址（IP地址＋端口）。

如果一个用户同时使用了两个会话，则每个会话对应的RTCP包都使用规范化名字CNAME（Canonical Name）。

与会者可以根据RTCP包中的CNAME来获取相关联的音频和视频，然后根据RTCP包中的计时信息(Network time protocol)来实现音频和视频的同步。

#### 翻译器和混合器

翻译器和混合器都是RTP级的中继系统。

翻译器用在通过IP多播不能直接到达的用户区，例如发送者和接收者之间存在防火墙。

当与会者能接收的音频编码格式不一样，比如有一个与会者通过一条低速链路接入到高速会议，这时就要使用混合器。

在进入音频数据格式需要变化的网络前，混合器将来自一个源或多个源的音频包进行重构，并把重构后的多个音频合并，采用另一种音频编码进行编码后，再转发这个新的RTP包。

从一个混合器出来的所有数据包要用混合器作为它们的同步源（SSRC，见RTP的封装）来识别，可以通过贡献源列表（CSRC表，见RTP的封装）可以确认谈话者。

### 相关概念

#### 流媒体

流媒体是指Internet上使用流式传输技术的连续时基媒体。当前在Internet上传输音频和视频等信息主要有两种方式：

- 下载传输
- 流式传输

下载情况下，用户需要先下载整个媒体文件到本地，然后才能播放媒体文件。在视频直播等应用场合，由于生成整个媒体文件要等直播结束，也就是用户至少要在直播结束后才能看到直播节目，所以用下载方式不能实现直播。

流式传输是实现流媒体的关键技术。使用流式传输可以边下载边观看流媒体节目。

由于Internet是基于分组传输的，所以接收端收到的数据包往往有延迟和乱序（流式传输构建在UDP上）。要实现流式传输，就是要从降低延迟和恢复数据包时序入手。

- 在发送端，为降低延迟，往往对传输数据进行预处理（降低质量和高效压缩）。

- 在接收端为了恢复时序，采用了接收缓冲；而为了实现媒体的流畅播放，则采用了播放缓冲。

使用接收缓冲，可以将接收到的数据包缓存起来，然后根据数据包的封装信息（如包序号和时戳等），将乱序的包重新排序，最后将重新排序了的数据包放入播放缓冲播放。

为什么需要播放缓冲呢？

容易想到，由于网络不可能很理想，并且对数据包排序需要处理时耗，我们得到排序好的数据包的时间间隔是不等的。如果不用播放缓冲，那么播放节目会很卡，这叫时延抖动。相反，使用播放缓冲，在开始播放时，花费几十秒钟先将播放缓冲填满（例如PPLIVE），可以有效地消除时延抖动，从而在不太损失实时性的前提下实现流媒体的顺畅播放。

**到目前为止,Internet 上使用较多的流式视频格式主要有以下三种:**

- RealNetworks 公司的RealMedia 
- Apple 公司的QuickTime 
- Microsoft 公司的Advanced Streaming Format (ASF) 

说到了流媒体数据包的封装信息（包序号和时戳等），这在后面的RTP封装中会有体现。另外，RealMedia这些流式媒体格式只是编解码有不同，但**对于RTP来说，它们都是待封装传输的流媒体数据，本质上没有什么不同。**


### RTP协议层次

RTP（实时传输协议），顾名思义它是用来提供实时传输的，因而可以看成是传输层的一个子层。

下图给出了流媒体应用中的一个典型的协议体系结构。

![流媒体应用的体系结构](images/11/流媒体应用的体系结构.jpg)

从图中可以看出，RTP被划分在传输层，它建立在UDP上。同UDP协议一样，为了实现其实时传输功能，RTP也有固定的封装形式。RTP用来为端到端的实时传输提供时间信息和流同步，但并不保证服务质量。服务质量由RTCP来提供。

不少人也把RTP归为应用层的一部分，这是从应用开发者的角度来说的。操作系统中的TCP/IP等协议栈所提供的是我们最常用的服务，而RTP的实现还是要靠开发者自己。因此从开发的角度来说，RTP的实现和应用层协议的实现没不同，所以可将RTP看成应用层协议。

### RTP 协议格式

一个协议的封装是为了满足协议的功能需求的。从前面提出的功能需求，可以推测出RTP封装中应该有同步源和时戳等字段，但更为完整的封装是什么样子呢？请看图。

![RTP的头部格式](images/11/RTP的头部格式.jpg)

- 版本号（V）：2比特，用来标志使用的RTP版本。

- 填充位（P）：1比特，如果该位置位，则该RTP包的尾部就包含附加的填充字节。

- 扩展位（X）：1比特，如果该位置位的话，RTP固定头部后面就跟有一个扩展头部。

- CSRC计数器（CC）：4比特，含有固定头部后面跟着的CSRC的数目。

- 标记位（M）：1比特,该位的解释由配置文档（Profile）来承担.

- 载荷类型（PT）：7比特，标识了RTP载荷的类型。

- 序列号（SN）：16比特，发送方在每发送完一个RTP包后就将该域的值增加1，接收方可以由该域检测包的丢失及恢复包序列。序列号的初始值是随机的。

- 时间戳：32比特，记录了该包中数据的第一个字节的采样时刻。在一次会话开始时，时间戳初始化成一个初始值。即使在没有信号发送时，时间戳的数值也要随时间而不断地增加（时间在流逝嘛）。时间戳是去除抖动和实现同步不可缺少的。

- 同步源标识符(SSRC)：32比特，同步源就是指RTP包流的来源。在同一个RTP会话中不能有两个相同的SSRC值。该标识符是随机选取的 RFC1889推荐了MD5随机算法。

- 贡献源列表（CSRC List）：0～15项，每项32比特，用来标志对一个RTP混合器产生的新包有贡献的所有RTP包的源。由混合器将这些有贡献的SSRC标识符插入表中。SSRC标识符都被列出来，以便接收端能正确指出交谈双方的身份。

### RTCP 协议格式

RTP需要RTCP为其服务质量提供保证，因此下面介绍一下RTCP的相关知识。

RTCP的主要功能是：

- 服务质量的监视与反馈、媒体间的同步

在RTP会话期间，各参与者周期性地传送RTCP包。RTCP包中含有已发送的数据包的数量、丢失的数据包的数量等统计资料；各参与者可以利用这些信息动态地改变传输速率，甚至改变有效载荷类型。

RTP和RTCP配合使用，它们能以有效的反馈和最小的开销使传输效率最佳化，因而特别适合传送网上的实时数据。

- 多播组中成员的标识



从之前的图上可以看到，RTCP也是用UDP来传送的，但RTCP封装的仅仅是一些控制信息，因而分组很短，所以可以将多个RTCP分组封装在一个UDP包中。

**RTCP有如下五种分组类型：**

|类型|缩写表示|用途|
|-|-|-|
|200|SR（Sender Report）|发送端报告|
|201|RR（Receiver Report）|接收端报告|
|202|SDES（Source Description Items）|源点描述|
|203|BYE|结束传输|
|204|APP|特定应用|

上述五种分组的封装大同小异，下面只讲述SR类型，而其它类型请参考RFC3550。

![RTCP的头部格式](images/11/RTCP的头部格式.jpg)

- 版本（V）：同RTP包头域。

- 填充（P）：同RTP包头域。

- 接收报告计数器（RC）：5比特，该SR包中的接收报告块的数目，可以为零。

- 包类型（PT）：8比特，SR包是200。

- 长度域（Length）：16比特，其中存放的是该SR包以32比特为单位的总长度减一。

- 同步源（SSRC）：SR包发送者的同步源标识符。与对应RTP包中的SSRC一样。

- NTP Timestamp（Network time protocol）SR包发送时的绝对时间值。NTP的作用是同步不同的RTP媒体流。

- RTP Timestamp：与NTP时间戳对应，与RTP数据包中的RTP时间戳具有相同的单位和随机初始值。

- Sender’s packet count：从开始发送包到产生这个SR包这段时间里，发送者发送的RTP数据包的总数. SSRC改变时，这个域清零。

- Senders octet count：从开始发送包到产生这个SR包这段时间里，发送者发送的净荷数据的总字节数（不包括头部和填充）。发送者改变其SSRC时，这个域要清零。

- 同步源n的SSRC标识符：该报告块中包含的是从该源接收到的包的统计信息。

- 丢失率（Fraction Lost）：表明从上一个SR或RR包发出以来从同步源n(SSRC_n)来的RTP数据包的丢失率。

- 累计的包丢失数目：从开始接收到SSRC_n的包到发送SR,从SSRC_n传过来的RTP数据包的丢失总数。

- 收到的扩展最大序列号：从SSRC_n收到的RTP数据包中最大的序列号，

- 接收抖动（Interarrival jitter）：RTP数据包接受时间的统计方差估计

- 上次SR时间戳（Last SR,LSR）：取最近从SSRC_n收到的SR包中的NTP时间戳的中间32比特。如果目前还没收到SR包，则该域清零。

- 上次SR以来的延时（Delay since last SR,DLSR）：上次从SSRC_n收到SR包到发送本报告的延时。

### RTP的会话过程

当应用程序建立一个RTP会话时，应用程序将确定一对目的传输地址。

目的传输地址由一个网络地址和一对端口组成，有两个端口：

- 一个给RTP包，
- 一个给RTCP包

使得RTP/RTCP数据能够正确发送。

RTP数据发向偶数的UDP端口，而对应的控制信号RTCP数据发向相邻的奇数UDP端口（偶数的UDP端口＋1），这样就构成一个UDP端口对。 RTP的发送过程如下，接收过程则相反。

- RTP协议从上层接收流媒体信息码流（如H.263），封装成RTP数据包；RTCP从上层接收控制信息，封装成RTCP控制包。

- RTP将RTP 数据包发往UDP端口对中偶数端口；RTCP将RTCP控制包发往UDP端口对中的接收端口。

### 常见的疑问

- 怎样重组乱序的数据包？

可以根据RTP包的序列号来排序。

- 怎样获得数据包的时序？

可以根据RTP包的时间戳来获得数据包的时序。

- 声音和图像怎么同步？

根据声音流和图像流的相对时间（即RTP包的时间戳），以及它们的绝对时间（即对应的RTCP包中的RTCP），可以实现声音和图像的同步。

- 接收缓冲和播放缓冲的作用

接收缓冲用来排序乱序了的数据包；播放缓冲用来消除播放的抖动，实现等时播放。


### 本节实验操作

- 打开magicjack_short_call样本文件
- 打开“电话”-“RTP”-“RTP 流”，回答以下问题：

|序号|源地址|源端口|目的地址|目的端口|载荷|分组|最大Delta（毫秒）|最大抖动（毫秒）|平均抖动（毫秒）|
|-|-|-|-|-|-|-|-|-|-|
|1|
|2|

- 使用其中“分析”功能，查看情形，并截图。


